In [9]:
import pandas as pd
import csv
import time

In [2]:
n = 4
file = './data/inst/knap_' + str(n) + '.inst.dat'
general_headers = ['ID', 'n', 'M']
all_headers = general_headers + [str(x) for x in range(1, 2*n + 1)]
used_cols = all_headers[3:]

all_headers
used_cols

['1', '2', '3', '4', '5', '6', '7', '8']

In [3]:
instances = pd.read_csv(file, sep=' ', names=all_headers, usecols=used_cols)
instances.head()

,1,2,3,4,5,6,7,8
0,18,114,42,136,88,192,3,223
1,55,29,81,64,14,104,52,222
2,89,196,18,62,57,34,69,112
3,34,169,23,152,62,44,2,224
4,67,145,74,111,87,139,65,243


In [4]:
[2**x for x in range(3)]

[1, 2, 4]

In [5]:
3 & 5

1

In [6]:
sum([1,2,3])

6

In [36]:
def bruteforce(n, M, pairs):
    keys = list(pairs.keys())
    max_num = 2**n - 1
    power2 = [2**x for x in range(n)]
    
    best = -1
    max_cost = 0
    max_weight = M
    final_keys = []
    start_time = time.clock()
    for i in range(1, max_num + 1):
        counter = 0
        cur_weight = 0
        cur_cost = 0
        used_keys = []
        
#         print(15*'=')
#         print('i:', i)
#         print(8*'-')
        for pow2 in power2:
#             print('pow2:', pow2)
            if pow2 & i:
#                 print('keys[counter]:', keys[counter])
                used_keys.append(keys[counter])
                cur_weight += keys[counter]
                cur_cost += pairs[keys[counter]]
            
            counter += 1
            
        if cur_weight <= M and cur_cost > max_cost:
#             print('best:', i)
#             print('max_cost:', cur_cost)
            best = i
            max_weight = cur_weight
            max_cost = cur_cost
            final_keys = used_keys
            
    end_time = time.clock()
    
    print(10*'=', 'BRUTEFORCE', 10*'=')
    print('elapsed time:', end_time - start_time)
    print('pairs:', pairs)
    print('keys:', keys)
    print('final_keys:', final_keys)
    print('M:', M)
    print('max_weight:', max_weight)
    print('max_cost:', max_cost)
    print('best:', best)
    
    return end_time - start_time

def fill_by(n, M, pairs, sorted_keys, sorted_by='cost'):
    max_cost = 0
    max_weight = 0
    final_keys = []
    
    start_time = time.clock()
    for weight in sorted_keys:
        if max_weight + weight > M:
            break
            
        max_weight += weight
        max_cost += pairs[weight]
        final_keys.append(weight)
        
    end_time = time.clock()
    
    print(10*'=', 'SORTED BY ' + sorted_by.upper() + 'S', 10*'=')
    print('elapsed time:', end_time - start_time)
    print('pairs:', pairs)
    print('keys:', sorted_keys)
    print('final_keys:', final_keys)
    print('M:', M)
    print('max_weight:', max_weight)
    print('max_cost:', max_cost)
    
    return end_time - start_time

In [37]:
with open('./data/inst/knap_4.inst.dat', 'r') as inst_file:
    reader = csv.reader(inst_file, delimiter=' ', skipinitialspace=True)
    
    bruteforce_time = []
    sorted_by_cost_time = []
    sorted_by_weight_time = []
    sorted_by_prop_time = []
    for row in reader:
        n = int(row[1])
        M = int(row[2])
        
        pairs = {}
        proportion_pairs = {}
        for i in range(0, n*2 - 1, 2):
            if int(row[i + 3]) in pairs.keys():
                print('There is a duplicate key!')
                
            # pair: weight - cost
            pairs[int(row[i + 3])] = int(row[i + 4])
            # used proportion: weight/cost
            proportion_pairs[int(row[i + 3])] = int(row[i + 3]) / int(row[i + 4])
            
#         print(pairs)
#         print('sort weights by costs in desc order')
        weights_by_cost = sorted(pairs, key=pairs.__getitem__, reverse=True)
#         print('weights:', weights_by_costs)
#         print(8*'-')
#         print('sort costs by weights in desc order')
#         print('costs:', [value for (key, value) in sorted(pairs.items(), reverse=True)])
        weights_by_weight = [key for (key, value) in sorted(pairs.items(), reverse=True)]
#         print('weights:', weights_by_weights)
#         print(18*'=')
        # ascending order
        weights_by_proportion = sorted(proportion_pairs, key=proportion_pairs.__getitem__)

        bruteforce_time.append(bruteforce(n, M, pairs))
        sorted_by_cost_time.append(fill_by(n, M, pairs, weights_by_cost, 'cost'))
        sorted_by_weight_time.append(fill_by(n, M, pairs, weights_by_weight, 'weight'))
        sorted_by_prop_time.append(fill_by(n, M, pairs, weights_by_proportion, 'proportion'))
        break

# print(bruteforce_time)

========== BRUTEFORCE ==========
elapsed time: 3.3783148410293506e-05
pairs: {88: 192, 18: 114, 3: 223, 42: 136}
keys: [88, 18, 3, 42]
final_keys: [18, 3, 42]
M: 100
max_weight: 63
max_cost: 473
best: 14
========== SORTED BY COSTS ==========
elapsed time: 2.5658086997282226e-06
pairs: {88: 192, 18: 114, 3: 223, 42: 136}
keys: [3, 88, 42, 18]
final_keys: [3, 88]
M: 100
max_weight: 91
max_cost: 415
========== SORTED BY WEIGHTS ==========
elapsed time: 1.2829041224904358e-06
pairs: {88: 192, 18: 114, 3: 223, 42: 136}
keys: [88, 42, 18, 3]
final_keys: [88]
M: 100
max_weight: 88
max_cost: 192
========== SORTED BY PROPORTIONS ==========
elapsed time: 2.5658086997282226e-06
pairs: {88: 192, 18: 114, 3: 223, 42: 136}
keys: [3, 18, 42, 88]
final_keys: [3, 18, 42]
M: 100
max_weight: 63
max_cost: 473


In [21]:
d = {'a': 3, 'b': [5,2], 'c': [4,6]}
print([key for (key, value) in sorted(d.items(), reverse=True)])
sorted(d, key=d.__getitem__, reverse=True)

['c', 'b', 'a']


TypeError: unorderable types: int() < list()